In [12]:
fn = "Abi-10-100m-Back.txt"
name, age, distance, stroke = fn.removesuffix(".txt").split("-")

In [13]:
name, age

('Abi', '10')

In [14]:
distance, stroke

('100m', 'Back')

In [15]:
import os

FOLDER = "swimdata/"

files = os.listdir(FOLDER)
if ".DS_Store" in files:
    files.remove(".DS_Store")

In [16]:
first = files[0]
print(first)
name, age, _, _ = first.removesuffix(".txt").split("-")
name, age

Katie-9-50m-Back.txt


('Katie', '9')

In [17]:
import DBcm

db_details = "CoachDB.sqlite3"

In [18]:
SQL_INSERT = """
    insert into swimmers
    (name, age)
    values
    (?, ?)
"""

In [19]:
with DBcm.UseDatabase(db_details) as db:
    db.execute(
        SQL_INSERT,
        (
            name,
            age,
        ),
    )

In [29]:
SQL = """
    select * from swimmers
"""

with DBcm.UseDatabase(db_details) as db:
    db.execute(SQL)
    results = db.fetchall()

results

[(25, 'Katie', 9),
 (26, 'Calvin', 9),
 (27, 'Hannah', 13),
 (28, 'Mike', 15),
 (29, 'Darius', 13),
 (30, 'Elba', 14),
 (31, 'Abi', 10),
 (32, 'Erika', 15),
 (33, 'Ruth', 13),
 (34, 'Maria', 9),
 (35, 'Alison', 14),
 (36, 'Emma', 13),
 (37, 'Blake', 15),
 (38, 'Lizzie', 14),
 (39, 'Ali', 12),
 (40, 'Chris', 17),
 (41, 'Aurora', 13),
 (42, 'Tasmin', 15),
 (43, 'Bill', 18),
 (44, 'Carl', 15),
 (45, 'Dave', 17),
 (46, 'Owen', 15)]

In [27]:
SQL = """
    delete from swimmers
"""

with DBcm.UseDatabase(db_details) as db:
    db.execute(SQL)

In [28]:
import os

import DBcm

db_details = "CoachDB.sqlite3"

FOLDER = "swimdata/"

files = os.listdir(FOLDER)
if ".DS_Store" in files:
    files.remove(".DS_Store")

SQL_SELECT = """
    select * from swimmers
    where name = ? and age = ?
"""

SQL_INSERT = """
    insert into swimmers
    (name, age)
    values
    (?, ?)
"""

with DBcm.UseDatabase(db_details) as db:
    for fn in files:
        name, age, _, _ = fn.removesuffix(".txt").split("-")
        db.execute(
            SQL_SELECT,
            (
                name,
                age,
            ),
        )
        if db.fetchall():
            continue
        db.execute(
            SQL_INSERT,
            (
                name,
                age,
            ),
        )

In [30]:
import os

import DBcm

db_details = "CoachDB.sqlite3"

FOLDER = "swimdata/"

files = os.listdir(FOLDER)
if ".DS_Store" in files:
    files.remove(".DS_Store")

SQL_SELECT = """
    select * from events
    where distance = ? and stroke = ?
"""

SQL_INSERT = """
    insert into events
    (distance, stroke)
    values
    (?, ?)
"""

with DBcm.UseDatabase(db_details) as db:
    for fn in files:
        _, _, distance, stroke = fn.removesuffix(".txt").split("-")
        db.execute(
            SQL_SELECT,
            (
                distance,
                stroke,
            ),
        )
        if db.fetchall():
            continue
        db.execute(
            SQL_INSERT,
            (
                distance,
                stroke,
            ),
        )

In [31]:
SQL = """
    select * from events
"""

with DBcm.UseDatabase(db_details) as db:
    db.execute(SQL)
    results = db.fetchall()

results

[(1, '50m', 'Back'),
 (2, '50m', 'Fly'),
 (3, '100m', 'Free'),
 (4, '200m', 'IM'),
 (5, '100m', 'Fly'),
 (6, '100m', 'Breast'),
 (7, '200m', 'Free'),
 (8, '50m', 'Free'),
 (9, '50m', 'Breast'),
 (10, '100m', 'Back'),
 (11, '400m', 'Free'),
 (12, '200m', 'Breast'),
 (13, '200m', 'Back')]

In [32]:
with DBcm.UseDatabase(db_details) as db:
    db.execute("pragma table_info(times)")
    results = db.fetchall()
results

[(0, 'swimmer_id', 'INTEGER', 1, None, 0),
 (1, 'event_id', 'INTEGER', 1, None, 0),
 (2, 'time', 'varchar(16)', 1, None, 0),
 (3, 'ts', 'timestamp', 0, 'current_timestamp', 0)]

In [ ]:
SQL_GET_SWIMMER = """
    select id from swimmers
    where name = ? and age = ?
"""

SQL_GET_EVENT = """
    select id from events
    where distance = ? and stroke = ?
"""

SQL_INSERT_SWIMTIMES = """
    insert into times
    (swimmer_id, event_id, time)
    values
    (?, ?, ?)
"""

with DBcm.UseDatabase(db_details) as db:
    for fn in files:
        name, age, distance, stroke = fn.removesuffix(".txt").split("-")
        db.execute(
            SQL_GET_SWIMMER,
            (
                name,
                age,
            ),
        )
        s_id = db.fetchone()[0]
        db.execute(
            SQL_GET_EVENT,
            (
                distance,
                stroke,
            ),
        )
        e_id = db.fetchone()[0]
        with open(FOLDER + fn) as sf:
            times = sf.read().strip().split(",")
            for t in times:
                db.execute(
                    SQL_INSERT_SWIMTIMES,
                    (
                        s_id,
                        e_id,
                        t,
                    ),
                )

In [35]:
SQL = """select count(*) from times"""
with DBcm.UseDatabase(db_details) as db:
    db.execute(SQL)
    result = db.fetchone()[0]
result

338